#  手动实现 PCA 主成分分析
本 Notebook 用于学习PCA，一步步拆解 PCA 算法的实现过程，并解释其背后的数学原理。

###  PCA 是什么？
- PCA（Principal Component Analysis）是一种常用的无监督降维技术。
- 它通过找到数据中方差最大的方向（主成分），将高维数据投影到低维空间。
- 目标：保留尽可能多的信息（方差），同时减少特征数量。

###  核心步骤回顾
1. 数据标准化（中心化）
2. 计算协方差矩阵
3. 特征值分解 → 得到主成分方向
4. 投影到前 n 个主成分空间

In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA

##  Step 1: 加载数据

In [3]:
# 加载鸢尾花数据集
data = load_iris()
X = data.data

print("原始数据 X shape:", X.shape)
print("前5个样本:\n", X[:5])

原始数据 X shape: (150, 4)
前5个样本:
 [[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]


##  Step 2: 数据标准化（中心化）

In [4]:
# 计算每个特征的均值
mean = np.mean(X, axis=0)

# 去中心化（减去均值）
X_centered = X - mean

print("数据中心化后的 shape:", X_centered.shape)
print("前5行样本:\n", X_centered[:5])

数据中心化后的 shape: (150, 4)
前5行样本:
 [[-0.74333333  0.446      -2.35866667 -0.99866667]
 [-0.94333333 -0.054      -2.35866667 -0.99866667]
 [-1.14333333  0.146      -2.45866667 -0.99866667]
 [-1.24333333  0.046      -2.25866667 -0.99866667]
 [-0.84333333  0.546      -2.35866667 -0.99866667]]


##  Step 3: 计算协方差矩阵
协方差矩阵描述了各特征之间的线性关系，PCA 利用它来找出数据变化最大的方向。

In [7]:
# 计算协方差矩阵
cov_matrix = np.cov(X_centered, rowvar=False)

print("cov shape:", cov_matrix.shape)
print("协方差矩阵:\n", cov_matrix) # 4个特征，就是4*4的对称矩阵

cov shape: (4, 4)
协方差矩阵:
 [[ 0.68569351 -0.03926846  1.27368233  0.5169038 ]
 [-0.03926846  0.18800403 -0.32171275 -0.11798121]
 [ 1.27368233 -0.32171275  3.11317942  1.29638747]
 [ 0.5169038  -0.11798121  1.29638747  0.58241432]]


##  Step 4: 特征值分解
协方差矩阵是一个对称矩阵，我们可以对其做特征值分解，其中：
- 特征值表示该方向上的方差大小（重要性）
- 特征向量就是我们想要找的主成分方向

In [8]:
# 特征值分解
eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)

print("特征值（排序前）:\n", eigenvalues)
print("特征向量（排序前）:\n", eigenvectors)

特征值（排序前）:
 [0.02368303 0.07852391 0.24224357 4.22484077]
特征向量（排序前）:
 [[ 0.31725455  0.58099728  0.65653988 -0.36158968]
 [-0.32409435 -0.59641809  0.72971237  0.08226889]
 [-0.47971899 -0.07252408 -0.1757674  -0.85657211]
 [ 0.75112056 -0.54906091 -0.07470647 -0.35884393]]


##  Step 5: 排序并选择主成分

In [11]:
# 按特征值从大到小排序
sorted_indices = np.argsort(eigenvalues)[::-1]
print("特征值排序索引：", sorted_indices)

# 选择前 n_components 个主成分
n_components = 2
components = eigenvectors[:, sorted_indices[:n_components]]

print("主成分（特征向量）shape:", components.shape)
print("主成分方向:\n", components)

特征值排序索引： [3 2 1 0]
主成分（特征向量）shape: (4, 2)
主成分方向:
 [[-0.36158968  0.65653988]
 [ 0.08226889  0.72971237]
 [-0.85657211 -0.1757674 ]
 [-0.35884393 -0.07470647]]


##  Step 6: 数据投影到主成分空间

In [12]:
# 将数据投影到主成分方向上
# 点积（np.dot）
X_pca = np.dot(X_centered, components)

print("降维后数据 shape:", X_pca.shape)
print("前5个样本投影结果:\n", X_pca[:5])

降维后数据 shape: (150, 2)
前5个样本投影结果:
 [[ 2.68420713  0.32660731]
 [ 2.71539062 -0.16955685]
 [ 2.88981954 -0.13734561]
 [ 2.7464372  -0.31112432]
 [ 2.72859298  0.33392456]]


## Step 7: 对比 sklearn 实现

In [ ]:
# 使用 sklearn 的 PCA 进行对比
sk_pca = PCA(n_components=2)
X_sk = sk_pca.fit_transform(X)

print("自定义 PCA 结果前5行:\n", X_pca[:5])
print("sklearn PCA 结果前5行:\n", X_sk[:5])

# PCA的数学解释

可以用于推导结论

### 一、 为什么要做协方差矩阵 + 特征值分解？
1. 目标函数
PCA 的目标是寻找一个单位向量 $ w $，使得数据在该方向上的投影方差最大： $$ \max_w \frac{1}{n} \sum_{i=1}^n (w^T x_i)^2 = w^T S w $$ 其中 $ S $ 是数据的协方差矩阵。

2. 拉格朗日乘子法求极值
引入约束 $ w^T w = 1 $，构造拉格朗日函数： $$ L(w) = w^T S w - λ(w^T w - 1) $$ 对 $ w $ 求导并令导数为零，得： $$ S w = λ w $$ 这就是标准的特征值问题。

3. 结论
最大方差对应的方向就是协方差矩阵的最大特征值对应的特征向量。
所有主成分就是按特征值大小排列的特征向量。
因此，PCA 的本质是：找到数据中方差最大的方向（主成分）。




### 二、为什么 PCA 的目标函数是最大化投影数据的方差？

$$ \max_w \frac{1}{n} \sum_{i=1}^n (w^T x_i)^2 = w^T S w $$

其中：

$ w $ 是一个单位向量（主成分方向）；
$ x_i $ 是原始数据点；
$ S $ 是数据的协方差矩阵。

**Insight1: PCA 的核心思想：保留最大信息**

PCA 的目标是降维，即在尽可能保留原始数据信息的前提下，把高维数据映射到低维空间中。
那么，“信息”在这里指的是什么？

---答： 方差。

- 数据的方差越大，说明该方向上包含的信息越多。
- 如果你把数据投影到一个方差很小的方向上，很多样本会挤在一起，无法区分。
- 所以 PCA 的目标是：找到一个方向（向量），使得数据在这个方向上的投影具有最大的方差。



**Insight2: 投影后的方差怎么计算？**

假设我们有以下数据：

数据矩阵 $ X \in \mathbb{R}^{n \times d} $：共 $ n $ 个样本，每个样本 $ d $ 维；

每个样本为行向量 $ x_i \in \mathbb{R}^d $；

我们选择一个单位向量 $ w \in \mathbb{R}^d $，表示我们要投影的方向。

将每个样本 $ x_i $ 投影到 $ w $ 上得到一个标量值： $$ z_i = w^T x_i $$

所有样本投影后的方差为： $$ \text{Var}(z) = \frac{1}{n} \sum_{i=1}^n z_i^2 - \left( \frac{1}{n} \sum_{i=1}^n z_i \right)^2 $$

如果数据已经中心化（即均值为0），那么第二项为0，所以： $$ \text{Var}(z) = \frac{1}{n} \sum_{i=1}^n (w^T x_i)^2 $$

这就是我们优化的目标函数： $$ \max_w \frac{1}{n} \sum_{i=1}^n (w^T x_i)^2 $$

**Insight3: 用矩阵形式简化表达式**

我们可以将上面的目标函数改写为矩阵形式：

$$ \frac{1}{n} \sum_{i=1}^n (w^T x_i)^2 = \frac{1}{n} \sum_{i=1}^n w^T x_i x_i^T w =w^T \left( \frac{1}{n} \sum_{i=1}^n x_i x_i^T \right) w = w^T S w $$

  其中： $$ S = \frac{1}{n} \sum_{i=1}^n x_i x_i^T = \frac{1}{n} X^T X $$ 就是数据的协方差矩阵（前提是数据已中心化）。